# Domains

In [1]:
import re
import pandas as pd

In [2]:
from dns.resolver import Resolver, NoAnswer

resolver = Resolver()
resolver.nameservers = ['8.8.8.8', '8.8.4.4']

In [3]:
import whois

In [4]:
df = pd.read_csv('data/domains.csv')

df.head()

,domain
0,6953727248492483.com
1,mylesb.ca
2,braithwaitecorp.com
3,mbraithwaite.com
4,mylesbraithwaite.ca


In [5]:
row = next(df.iterrows())[1]

row

domain    6953727248492483.com
Name: 0, dtype: object

In [6]:
def registrar(row):
    w = whois.whois(row.domain)
    
    try:
        return w['registrar']
    except:
        None

# registrar(row)

In [7]:
#df['registrar'] = df.apply(lambda row: registrar(row), axis=1)

#df.tail()

In [8]:
def nameserver(row):
    try:
        answer = resolver.query(row.name, 'ns')
    except NoAnswer:
        return None
        
    ns_value = ' | '.join(ns.to_text() for ns in answer.rrset.items)
    return ns_value

nameserver(row)

AttributeError: 'int' object has no attribute 'is_absolute'

In [ ]:
df['dns'] = df.apply(lambda row: nameserver(row), axis=1)

df.tail()

In [ ]:
def mx_record(row):
    try:
        answer = resolver.query(row.domain, 'mx')
    except NoAnswer:
        return None
    
    mx_value = ' | '.join(ns.to_text() for ns in answer.rrset.items)
    return mx_value

mx_record(row)

In [ ]:
df['mx'] = df.apply(lambda row: mx_record(row), axis=1)

df.tail()

In [ ]:
def xmpp_records(row):
    try:
        xmpp_client_domain = '_xmpp-client._tcp.{}'.format(row.domain)
        xmpp_client_answer = resolver.query(xmpp_client_domain, 'srv')
        
        xmpp_server_domain = '_xmpp-server._tcp.{}'.format(row.domain)
        xmpp_server_answer = resolver.query(xmpp_server_domain, 'srv')
    except NoAnswer:
        return None
    
    xmpp_client_value = ' | '.join(record.to_text() for record in xmpp_client_answer.rrset.items)
    xmpp_server_value = ' | '.join(record.to_text() for record in xmpp_server_answer.rrset.items)
    return ' | '.join([xmpp_client_value, xmpp_server_value])

xmpp_records(row)

In [ ]:
df['xmpp'] = df.apply(lambda row: xmpp_records(row), axis=1)

df.tail()

In [ ]:
def dkim_record(row):
    try:
        answer = resolver.query('default._domainkey.{}'.format(row.domain), 'txt')
    except NoAnswer:
        return None
    
    value = answer.rrset.items[0].to_text()
    return value

dkim_record(row)

In [ ]:
df['dkim'] = df.apply(lambda row: dkim_record(row), axis=1)

df.tail()

In [ ]:
def dmarc_record(row):
    try:
        answer = resolver.query('_dmarc.{}'.format(row.domain), 'txt')
    except NoAnswer:
        return None
    
    value = answer.rrset.items[0].to_text()
    return value

dmarc_record(row)

In [ ]:
df['dmarc'] = df.apply(lambda row: dmarc_record(row), axis=1)

df.tail()

In [ ]:
def spf_record(row):
    try:
        answer = resolver.query(row.domain, 'txt')
    except NoAnswer:
        return None
    
    for record in answer.rrset.items:
        if 'spf' in record.to_text():
            return record.to_text()
    
    return None

spf_record(row)

In [ ]:
df['spf'] = df.apply(lambda row: spf_record(row), axis=1)

df.tail()

In [ ]:
def autoconfig_record(row):
    try:
        answer = resolver.query('autoconfig.{}'.format(row.domain), 'cname')
    except NoAnswer:
        return None
    
    value = answer.rrset.items[0].to_text()
    return value

autoconfig_record(row)

In [ ]:
df['autoconfig'] = df.apply(lambda row: autoconfig_record(row), axis=1)

df.tail()

In [ ]:
def autodiscover_record(row):
    try:
        answer = resolver.query('autodiscover.{}'.format(row.domain), 'cname')
    except NoAnswer:
        return None
    
    value = answer.rrset.items[0].to_text()
    return value

autodiscover_record(row)

In [ ]:
df['autodiscover'] = df.apply(lambda row: autodiscover_record(row), axis=1)

df.tail()

In [ ]:
df.to_csv('domains.csv')